# Supervised ML for anomaly detection in IOT to enahnce network security
## Part 4 - DATA TUNING

The IoT-23 dataset is a collection of network traffic from Internet of Things (IoT) devices. It includes 20 malware captures executed in IoT devices, and 3 hotspot captures for benign IoT devices traffic12. The 3 hotspot captures are not being included in the data cleaning because this feature was not considered relevant for the specific analysis being performed.

In this notebook, we load the raw dataset file and implement initial cleaning to prepare it for the next processing phase.

> **INPUT:** downloaded the raw dataset file from its original source. <br>
> **OUTPUT:** a cleaned version of the dataset stored to an intermediate csv file.

***

In [ ]:
# Import necessary libraries and modules
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC

In [ ]:
# Read the dataset
data_df = pd.read_csv('../data/processed/iot23_combined_processed.csv', index_col=0)

In [ ]:
# Split data into independent and dependent variables
data_X = data_df.drop("label", axis=1)
data_y = data_df["label"]

In [ ]:
# Initialize the model
model=SVC()

# Set hyperparameters
parameters={
    'C':[0.1, 1, 10, 100, 1000],
    'gamma':[1, 0.1, 0.01, 0.001, 0.0001],
}

# Initialize cross validation method
cross_validation_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Initialize tuning process
grid = GridSearchCV(
    estimator=model, 
    param_grid=parameters, 
    scoring=['f1','precision','recall'],
    cv=cross_validation_folds,
    verbose=100,
    refit="precision")

# Train the model
grid.fit(data_X, data_y)

# Store performance metrics
results = pd.DataFrame(index=["SVC Base", "SVC Tuned"], columns=["Recall", "Precision", "F1"])
results.iloc[0] = [0.999906, 0.995403, 0.997649] # Results obtained from previous phase
results.iloc[1] = [grid.cv_results_['mean_test_recall'][grid.best_index_], grid.cv_results_['mean_test_precision'][grid.best_index_], grid.cv_results_['mean_test_f1'][grid.best_index_]]
print ("Best Parameters: {}".format(grid.best_params_))

In [ ]:
# Check and compare results
results